In [11]:
# breadboard_repo_path =  r'D:\Fermidata1\enrico\breadboard-python-client\\'
breadboard_repo_path =  r'C:\Users\Alex\Dropbox (MIT)\lab side projects\control software\breadboard-python-client\\'
import sys
sys.path.insert(0, breadboard_repo_path)
from breadboard import BreadboardClient
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json') # enter your path to the API_config
from IPython.display import Image
import ipywidgets as widgets
import qgrid
import pandas as pd
import time
import warnings
from measurement_directory import *
from numpy import nan
from ipywidgets import interact
import os
from image_watchdog import getFileList
import json
import numpy as np

def get_newest_df(watchfolder, optional_column_names = []):
    files, _ = getFileList(watchfolder)
    run_ids = [run_id_from_filename(file) for file in files]
    return bc.get_runs_df_from_ids(run_ids, optional_column_names = optional_column_names)

measurement_menu = widgets.Dropdown(options = todays_measurements(),
                                   description = 'today\'s measurements',
                                   style = {'description_width': 'initial'})
load_measurement_button = widgets.Button(description = 'select measurement')
def load_measurement(button):
    global watchfolder
    watchfolder = measurement_directory(measurement_name = measurement_menu.value)
    print(watchfolder)
load_measurement_button.on_click(load_measurement)
widgets.HBox([measurement_menu, load_measurement_button])

## Run the next two cells to load the log from one of today's runs.

In [9]:
def edit_image_log(optional_column_names = []):
    df = get_newest_df(watchfolder, optional_column_names = optional_column_names)
#for testing
#     df = bc.get_runs_df_from_ids([219153, 219151], 
#                                  optional_column_names = optional_column_names)
    for column in optional_column_names:
        if column not in df.columns:
            df[column] = nan
    
    col_opts = { 'editable': False }
    col_defs = {}
    for col_name in optional_columns + ['badshot', 'notes']:
        col_defs[col_name] = { 'editable': True }
    col_defs['badshot']['ColumnWidth'] = 50
    qgrid_widget = qgrid.show_grid(df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100},
                                  column_options=col_opts, column_definitions=col_defs)
    return qgrid_widget

def save_image_log(event, qgrid_widget):
    row_idx = int(event['index'])
    df = qgrid_widget.get_changed_df()
    df_updated_row = df.loc[row_idx]
    updated_row_dict = {}
    for key in df_updated_row.index:
        if key == 'notes':
            continue
        elif key == 'badshot':
            updated_row_dict[key] = bool(df_updated_row.loc[key])
        else:
            if np.isnan(df_updated_row.loc[key]):
                continue #don't upload nan's to breadboard
            elif isinstance(df_updated_row.loc[key], np.integer):
                updated_row_dict[key] = int(df_updated_row.loc[key])
            elif isinstance(df_updated_row.loc[key], np.float):
                updated_row_dict[key] = float(df_updated_row.loc[key])
            else:
                updated_row_dict[key] = df_updated_row.loc[key]
    run_id = df_updated_row['run_id']
    run_dict = bc._send_message(
            'get', '/runs/' + str(run_id) + '/').json()
    run_dict['notes'] = df_updated_row['notes']
    run_dict['parameters'].update(updated_row_dict)
    payload = json.dumps(run_dict)
#     print(payload)
    resp = bc._send_message('put', '/runs/' + str(run_id) + '/', data=payload)
    qgrid_widget.close()
    qgrid_widget = edit_image_log(optional_columns)
    qgrid_widget.on('cell_edited',save_image_log)
    display(qgrid_widget)

# image_viewer = widgets.Output(layout={'border': '1px solid black'})
# def show_selected_image(event, qgrid_widget):
#     image_viewer.clear_output()
#     row_idx = int(event['new'][0])
#     df = pd.read_csv(filename)
#     raw_image_filename = df.loc[row_idx, 'filename0']
#     filepath = measurement_dir + '\\' + raw_image_filename.replace('.spe', '.jpg')
#     with image_viewer:
#         if os.path.exists(filepath) and '.jpg' in filepath:
#             pil_img = Image(filepath)
#             display(pil_img)
#         else:
#             display('no jpg preview at ' + filepath)

optional_columns = []
textbox = widgets.Text(
    value='',
    description='editable column name',
    disabled=False,
    style = {'description_width': 'initial'}
)
add_column_button = widgets.Button(description = 'add')
def add_column(button):
    global optional_columns
    if len(textbox.value) > 0:
        optional_columns = list(set().union(['manual_' + textbox.value],optional_columns))
    print(optional_columns)
add_column_button.on_click(add_column)
clear_column_button = widgets.Button(description = 'clear column')
def clear_column(button):
    global optional_columns
    if len(optional_columns) > 0:
        optional_columns.pop()
    print(optional_columns)
clear_column_button.on_click(clear_column)
widgets.HBox([textbox, add_column_button, clear_column_button])

In [4]:
%matplotlib notebook
qgrid_widget = edit_image_log(optional_columns)
qgrid_widget.on('cell_edited',save_image_log)
# qgrid_widget.on('selection_changed', show_selected_image)
display(qgrid_widget)
# display(image_viewer)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [ ]:
df = bc.get_runs_df_from_ids([219153, 219152], optional_column_names=['manual_test2'])
# df['test_variable'] = nan
# df['editable_variables'] = [['test_variable']] * len(df)
# df.loc[0,'test_variable']
df

In [ ]:
'manual' in 'manual_test1'